In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
con = sqlite3.connect("../data/juans_history-2022-10-14")
#visits = pd.read_sql_query("SELECT * from visits", con)
urls = pd.read_sql_query("SELECT * from urls", con)
visits_with_urls = pd.read_sql_query(("SELECT vs.transition, vs.visit_duration, ur.url, ur.title, vs.visit_time FROM 'visits' vs JOIN 'urls' ur ON ur.id = vs.url"), con)
visits_with_urls['visit_duration'] = visits_with_urls['visit_duration'].div(1000000).round(2)
visits_with_urls.head(20) #visit duration in microseconds

,transition,visit_duration,url,title,visit_time
0,805306375,0.00,https://mail.sfu.ca/owa/,Mail - juan_antonio_gonzalez@sfu.ca,13303114120267367
1,805306375,0.00,https://mail.sfu.ca/owa/,Mail - juan_antonio_gonzalez@sfu.ca,13303162217955155
2,805306375,0.00,https://mail.sfu.ca/owa/,Mail - juan_antonio_gonzalez@sfu.ca,13303245851920431
3,268435457,0.00,https://canvas.sfu.ca/login/cas,SFU Multi-Factor Authentication - CAS – Centra...,13303247908193105
4,-1610612735,3.31,https://cas.sfu.ca/cas/login?service=https%3A%...,SFU Multi-Factor Authentication - CAS – Centra...,13303247908193105
5,268435457,0.00,https://docs.google.com/,Google Docs,13303247908391612
6,-2147483647,0.00,https://docs.google.com/document/?usp=direct_u...,Google Docs,13303247908391612
7,-1610612735,0.00,https://docs.google.com/document/u/0/?usp=dire...,Google Docs,13303247908391612
8,805306368,1811.92,https://docs.google.com/document/u/0/,Google Docs,13303247913831293
9,268435463,0.00,https://canvas.sfu.ca/login/cas?ticket=ST-7773...,Dashboard,13303247914367631


In [3]:
import re

# Match https://*/
regex_extract_domain = re.compile('^[^/]+://([^/]+/?[^/\?]*)')

# Match file:///* and data:image/*
regex_extract_domain_temp = re.compile('^([^/]+:/+[^/]+/?[^/\?]*|data:[^/]+)')
def extract_domain_with_regex(url: str):
  m = regex_extract_domain.match(url)
  try:
    domain = m.group(1)
  except AttributeError:
    m2 = regex_extract_domain_temp.match(url)

    try:
      domain = m2.group(1)
    except AttributeError:
      domain = "ERROR"

  return domain

In [4]:
def extract_domain_with_string_funcs(url:str):
  start = url.find('//') + 2
  end = url.find('/', start)
  return url[start:end]

In [5]:
def extract_domain(url:str):
  return extract_domain_with_string_funcs(url)

In [6]:
transition_dict = {
    0:"transition_link",
    1:"typed",
    2: "recommendation",
    3: "auto_subframe",
    4: "manual_subframe",
    5: "generated",
    6:"start_page",
    7:"form-submit",
    8:"reload",
    9:"keyword",
    10:"keyword_generated"}

def extr_mask(df_visits):
    df_visits['core-mask'] = df_visits['transition'] & 0xff
    df_visits['mask'] = df_visits['core-mask'].replace(transition_dict, inplace=False)
    return df_visits

In [7]:
map_domains = {
  'google.ca/maps': 'google.com/maps',
  'google.ca': 'google.com',
}

def process_domain(url:str):
  domain : str = extract_domain(url)
  domain = domain.strip('/')
  domain = domain.removeprefix('www.').removeprefix('www2.')

  if (domain in map_domains):
    return map_domains[domain]
  else:
    return domain
  
s = 5.0
#visits_with_urls = extr_mask(visits_with_urls)
df_visits = visits_with_urls[(visits_with_urls['visit_duration'] > s)] #Websites visited for more than s seconds
df_visits['domain'] = df_visits['url'].apply(process_domain)
df_visits

C:\Users\juang\AppData\Local\Temp/ipykernel_13084/4047078387.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_visits['domain'] = df_visits['url'].apply(process_domain)


,transition,visit_duration,url,title,visit_time,domain
8,805306368,1811.92,https://docs.google.com/document/u/0/,Google Docs,13303247913831293,docs.google.com
10,-1610612729,22.93,https://canvas.sfu.ca/?login_success=1,Dashboard,13303247914367631,canvas.sfu.ca
14,-1610612736,1693.75,https://canvas.sfu.ca/courses/70194/pages/lect...,Lecture Notes (Pre and Post Slides ): CMPT354 ...,13303247941028634,canvas.sfu.ca
109,805306368,655.02,https://canvas.sfu.ca/,Dashboard,13303249634776313,canvas.sfu.ca
113,805306368,549.86,https://docs.google.com/document/d/1X_rxKMLgxK...,CMPT 354 - Google Docs,13303249739950253,docs.google.com
...,...,...,...,...,...,...
20754,805306368,21.27,https://www.google.com/search?q=weather&oq=wea...,weather - Google Search,13310844051137266,google.com
20762,-1610612729,7.62,https://canvas.sfu.ca/?login_success=1,Dashboard,13310844080872011,canvas.sfu.ca
20763,805306368,9.70,https://canvas.sfu.ca/courses/72256,MACM201 D100 Discrete Mathematics II,13310844088495566,canvas.sfu.ca
20783,805306368,10.41,https://support.google.com/chrome/answer/10248...,Export your data from Chrome - Google Chrome Help,13310844700552656,support.google.com


In [8]:
df_count = df_visits.groupby('domain')['domain'].size().reset_index(name='count')
top_50_filtered = df_count.sort_values("count", ascending=False)[0:50]
top_50_filtered.to_csv('../output/top_50_filtered_juan.csv')
top_50_filtered

,domain,count
89,google.com,734
31,canvas.sfu.ca,451
227,youtube.com,186
114,localhost:8888,146
48,coursys.sfu.ca,125
97,ign.com,80
113,localhost:3000,75
194,tab.gladly.io,63
62,docs.google.com,57
185,stackoverflow.com,39


In [14]:
relevant_fields =       [ 'visit_time', 'visit_duration', 'domain','url', 'title']

df_top_50 = df_visits[relevant_fields].join(top_50_filtered.set_index('domain').drop(columns=['count']), on='domain', how='inner')
df_top_50

,visit_time,visit_duration,domain,url,title
8,13303247913831293,1811.92,docs.google.com,https://docs.google.com/document/u/0/,Google Docs
113,13303249739950253,549.86,docs.google.com,https://docs.google.com/document/d/1X_rxKMLgxK...,CMPT 354 - Google Docs
816,13303282244480958,21.42,docs.google.com,https://docs.google.com/document/u/0/,Google Docs
874,13303282265900260,9.40,docs.google.com,https://docs.google.com/document/d/19SrcjlNzkN...,Milestone 3 - Google Docs
877,13303282275302659,39.22,docs.google.com,https://docs.google.com/document/u/0/,Google Docs
...,...,...,...,...,...
17166,13310112736353872,795.88,wildriftfire.com,https://www.wildriftfire.com/guide/wukong,Wukong :: Wild Rift Build Guide (Patch 3.4a) :...
19306,13310583078257448,7.99,wildriftfire.com,https://www.wildriftfire.com/tier-list,Wild Rift Tier List (Patch 3.4a) :: All Roles
19308,13310583193158161,11.33,wildriftfire.com,https://www.wildriftfire.com/tier-list,Wild Rift Tier List (Patch 3.4a) :: All Roles
19346,13310584655611435,5.65,wildriftfire.com,https://www.wildriftfire.com/tier-list,Wild Rift Tier List (Patch 3.4a) :: All Roles


In [16]:
#Not really important, just wanted to check the time at which page was visited 

from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo

USER_TIMEZONE = timezone(offset=-timedelta(hours=7))
TIMEZONE_STRING = 'UTC-07:00'
WINDOWS_EPOCH_MICROSECS      = -11644473600000 * 1000
TIME_TO_START_VIEWING_DATA  =  datetime(2022, 9, 22, tzinfo=USER_TIMEZONE)


visit_time_in_ns = (df_top_50['visit_time'] + WINDOWS_EPOCH_MICROSECS) * 1000
df_top_50['visit_time_epoch'] = pd.to_datetime(visit_time_in_ns, unit='ns', utc=True).map(lambda x: x.tz_convert(TIMEZONE_STRING))
df_top_50

,visit_time,visit_duration,domain,url,title,visit_time_epoch
8,13303247913831293,1811.92,docs.google.com,https://docs.google.com/document/u/0/,Google Docs,2022-07-25 11:38:33.831293-07:00
113,13303249739950253,549.86,docs.google.com,https://docs.google.com/document/d/1X_rxKMLgxK...,CMPT 354 - Google Docs,2022-07-25 12:08:59.950253-07:00
816,13303282244480958,21.42,docs.google.com,https://docs.google.com/document/u/0/,Google Docs,2022-07-25 21:10:44.480958-07:00
874,13303282265900260,9.40,docs.google.com,https://docs.google.com/document/d/19SrcjlNzkN...,Milestone 3 - Google Docs,2022-07-25 21:11:05.900260-07:00
877,13303282275302659,39.22,docs.google.com,https://docs.google.com/document/u/0/,Google Docs,2022-07-25 21:11:15.302659-07:00
...,...,...,...,...,...,...
17166,13310112736353872,795.88,wildriftfire.com,https://www.wildriftfire.com/guide/wukong,Wukong :: Wild Rift Build Guide (Patch 3.4a) :...,2022-10-12 22:32:16.353872-07:00
19306,13310583078257448,7.99,wildriftfire.com,https://www.wildriftfire.com/tier-list,Wild Rift Tier List (Patch 3.4a) :: All Roles,2022-10-18 09:11:18.257448-07:00
19308,13310583193158161,11.33,wildriftfire.com,https://www.wildriftfire.com/tier-list,Wild Rift Tier List (Patch 3.4a) :: All Roles,2022-10-18 09:13:13.158161-07:00
19346,13310584655611435,5.65,wildriftfire.com,https://www.wildriftfire.com/tier-list,Wild Rift Tier List (Patch 3.4a) :: All Roles,2022-10-18 09:37:35.611435-07:00


In [11]:
# https://stackoverflow.com/a/65784434
relevant_fields =       ['visit_count', 'url', 'title', 'domain']
relevant_fields_alias = ['visit_count', 'sample_url', 'sample_title']

df_grouped_count = df_visits[relevant_fields].groupby('domain').agg(np.sum)
top_50 = df_grouped_count.sort_values('visit_count', ascending=False)[0:50]

top_50.to_csv('../output/top_50_plain_juan.csv')

df_grouped_count__with_first = df[relevant_fields].groupby('domain').agg({k: np.sum if k == 'visit_count' else 'first' for k in relevant_fields})
top_50__with_first = df_grouped_count__with_first[relevant_fields[0:3]].sort_values('visit_count', ascending=False)[0:50]

top_50__with_first.to_csv('../output/top_50_juan.csv', header=relevant_fields_alias)

NameError: name 'df_visits' is not defined

In [10]:
top_50__with_first

,visit_count,url,title
domain,,,
docs.google.com,3916,https://docs.google.com/document/u/0/,Google Docs
youtube.com,2996,https://www.youtube.com/,(1) YouTube
google.com,2792,https://www.google.com/maps,Google Maps
ggbaker.ca,1263,https://ggbaker.ca/data-science/,CMPT 353 Lecture Notes
canvas.sfu.ca,913,https://canvas.sfu.ca/login/cas,SFU Multi-Factor Authentication - CAS – Centra...
localhost:3000,452,http://localhost:3000/,Save Inn
figma.com,368,https://www.figma.com/file/bMYAFkTYFAYASKGl8J3...,SaveInn – Figma
localhost:8888,338,http://localhost:8888/tree?token=922d8950ee426...,Home Page - Select or create a notebook
ign.com,337,https://www.ign.com/wikis/pokemon-omega-ruby-a...,Mauville City Pokemon Gym - Pokemon Omega Ruby...
